# Lecture 2: Linear Regression

(Based on the notebook provided by Marek Rei)

Loading the necessary libraries and the dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

data = pd.read_csv('data/country-stats.csv')
data.head()

## Finding a Good Model

We can try a very basic model: $y=x$  
If $x$ and $y$ are very highly correlated and in the same range, then this can actually give a reasonable result.

In [ ]:
plt.scatter(data["Estimated Control of Corruption (scale -2.5 to 2.5)"], 
            data["Estimated Government Effectiveness (scale -2.5 to 2.5)"])
plt.title('Estimated Corruption vs Government Effectiveness')
plt.xlabel("Estimated Control of Corruption")
plt.ylabel("Estimated Government Effectiveness")
x_sample = np.linspace(-2.5, 2.5, 100)
y = x_sample
plt.plot(x_sample, y, color='orange')
plt.show()

However, for most practical cases this is not true, with $x$ and $y$ varying along very different ranges.

In [ ]:
plt.scatter(data["Estimated Control of Corruption (scale -2.5 to 2.5)"], 
            data["Urban Population (%)"])
plt.title('Estimated Corruption vs Urban Population')
plt.xlabel("Estimated Control of Corruption")
plt.ylabel("Urban Population (%)")
x_sample = np.linspace(-2.5, 2.5, 100)
y = x_sample
plt.plot(x_sample, y, color='orange')
plt.show()

Let's try a slightly more flexible model: $y=ax$

In [ ]:
plt.scatter(data["Estimated Control of Corruption (scale -2.5 to 2.5)"], 
            data["Urban Population (%)"])
plt.title('Estimated Corruption vs Urban Population')
plt.xlabel("Estimated Control of Corruption")
plt.ylabel("Urban Population (%)")

model = LinearRegression(fit_intercept=False)
X = data["Estimated Control of Corruption (scale -2.5 to 2.5)"].values.reshape(-1,1)
Y = data["Urban Population (%)"]
model.fit(X, Y)

x_sample = np.linspace(-2.5, 2.5, 100)
plt.plot(x_sample, model.predict(x_sample.reshape(-1,1)), color='orange')
plt.show()

How about $y = b$

In [ ]:
from sklearn.linear_model import LinearRegression
plt.scatter(data["Estimated Control of Corruption (scale -2.5 to 2.5)"], 
            data["Urban Population (%)"])
plt.title('Estimated Corruption vs Urban Population')
plt.xlabel("Estimated Control of Corruption")
plt.ylabel("Urban Population (%)")

model = LinearRegression(fit_intercept=True)
X = np.zeros((len(data["Urban Population (%)"]), 1), dtype=float)
Y = data["Urban Population (%)"]
model.fit(X, Y)

x_sample = np.linspace(-2.5, 2.5, 100)
plt.plot(x_sample, model.predict(x_sample.reshape(-1,1)), color='orange')
plt.show()

Putting these together, we get the simple linear regression model: $y = ax + b$

In [ ]:
from sklearn.linear_model import LinearRegression
plt.scatter(data["Estimated Control of Corruption (scale -2.5 to 2.5)"], 
            data["Urban Population (%)"])
plt.title('Estimated Corruption vs Urban Population')
plt.xlabel("Estimated Control of Corruption")
plt.ylabel("Urban Population (%)")

model = LinearRegression(fit_intercept=True)
X = data["Estimated Control of Corruption (scale -2.5 to 2.5)"].values.reshape(-1,1)
Y = data["Urban Population (%)"]
model.fit(X, Y)

x_sample = np.linspace(-2.5, 2.5, 100)
plt.plot(x_sample, model.predict(x_sample.reshape(-1,1)), color='orange')
plt.show()

## Optimizing Linear Regression: Gradient Descent

Let's look at a linear regression model predicting enrolment rate from GDP.

First, let's define a helper function to visualise a line ($y = ax + b$) that has been fit to a dataset (X, Y):

In [ ]:
def plot_simple_linear_regression(X, Y, a, b, title, xlabel, ylabel):
    plt.scatter(X, Y)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    x_sample = np.linspace(np.min(X), np.max(X), 100)
    plt.plot(x_sample, x_sample*a + b, color='orange')
    plt.show()

A verbose version of gradient descent, iterating over epochs and each datapoint:

In [ ]:
X = data["GDP per Capita (PPP USD)"].values
Y = data["Enrolment Rate, Tertiary (%)"].values

a = 0.0
b = 0.0
learning_rate = 1e-11


for epoch in range(10):
    update_a = 0.0
    update_b = 0.0
    error = 0.0
    for i in range(len(Y)):
        y_predicted = a * X[i] + b
        update_a += (y_predicted - Y[i])*X[i]
        update_b += (y_predicted - Y[i])
        error += np.square(y_predicted - Y[i])
    a = a - learning_rate * update_a
    b = b - learning_rate * update_b
    rmse = np.sqrt(error / len(Y))
    print("a: {:.4f}\tb: {:.9f}\tRMSE: {:.4f}".format(a, b, rmse))
    #print(f"a: {round(a, 4)}\tb: {round(b, 9)}\tRMSE: {round(rmse, 4)}")
plot_simple_linear_regression(X, Y, a, b, 
                              "GDP vs Enrolment Rate", 
                              "GDP per Capita (PPP USD)", 
                              "Enrolment Rate, Tertiary (%)")

A more compact version, using python vector operations:

In [ ]:
X = data["GDP per Capita (PPP USD)"].values
Y = data["Enrolment Rate, Tertiary (%)"].values

a = 0.0
b = 0.0
learning_rate = 1e-11

for epoch in range(10):
    y_predicted = a * X + b
    a = a - learning_rate * ((y_predicted - Y)*X).sum()
    b = b - learning_rate * (y_predicted - Y).sum()
    rmse = np.sqrt(np.square(y_predicted - Y).mean())
    print("RMSE: " + str(rmse))

plot_simple_linear_regression(X, Y, a, b, 
                              "GDP vs Enrolment Rate", 
                              "GDP per Capita (PPP USD)", 
                              "Enrolment Rate, Tertiary (%)")

## Optimizing Linear Regression: The Analytical Solution with Scikit-Learn

Using scikit-learn to calculate the analytical least squares solution:

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True)
X = data["GDP per Capita (PPP USD)"].values.reshape(-1,1)
Y = data["Enrolment Rate, Tertiary (%)"]
model.fit(X, Y)

mse = np.square(Y - model.predict(X)).mean()
print("RMSE: " + str(np.sqrt(mse)))

plt.scatter(data["GDP per Capita (PPP USD)"], 
            data["Enrolment Rate, Tertiary (%)"])
plt.title('GDP vs Enrolment Rate')
plt.xlabel("GDP per Capita (PPP USD)")
plt.ylabel("Enrolment Rate, Tertiary (%)")

x_sample = np.linspace(0, 80000, 100)
plt.plot(x_sample, 
         model.predict(x_sample.reshape(-1,1)), 
         color='orange')
plt.show()

## Multiple Linear Regression

Predicting enrolment rate using all the available variables in the dataset.  
We have to exclude the country name (because it's text and we can only handle numerical features at the moment) and the enrolment rate itself (because using that would be cheating).

We can't really visualize an 11-dimensional graph, so we'll still project onto the same GDP vs Enrolment Rate graph, but the predictions will be made based on all 11 features.

In [ ]:
model = LinearRegression(fit_intercept=True)
X = data.copy().drop(["Country Name", "Enrolment Rate, Tertiary (%)"], axis=1)
Y = data["Enrolment Rate, Tertiary (%)"]

model.fit(X, Y)

mse = np.square(Y - model.predict(X)).mean()
print("RMSE: " + str(np.sqrt(mse)))

plt.scatter(data["GDP per Capita (PPP USD)"], 
            data["Enrolment Rate, Tertiary (%)"])
plt.title('GDP vs Enrolment Rate')
plt.xlabel("GDP per Capita (PPP USD)")
plt.ylabel("Enrolment Rate, Tertiary (%)")

x_pred = data["GDP per Capita (PPP USD)"]
plt.plot(x_pred, model.predict(X), '.', color='orange')
plt.show()

We can go in and inspect each of the learned coefficients in the model.

In [ ]:
headers=list(X)
coefficients = []
for i in range(len(headers)):
    coefficients.append({"Property": headers[i], 
                         "coefficient": model.coef_[i]})
pd.DataFrame(coefficients)

While interesting, these values are not comparable, because the range of each of the input features is very different and this is reflected in the coefficients.  
We can standardize the input features before model fitting, which will make the coefficients comparable.

An example of applying standardization to features:

In [ ]:
Z = pd.DataFrame(data, columns=["GDP per Capita (PPP USD)"])
Z_scaled = preprocessing.scale(Z)

asd = pd.concat([Z, pd.DataFrame(Z_scaled)], axis=1)
asd.columns = ["Z", "Z_scaled"]
asd.head()

Now we can apply it to our features before fitting the model:

In [ ]:
model = LinearRegression(fit_intercept=True)
X = data.copy().drop(["Country Name", "Enrolment Rate, Tertiary (%)"], axis=1)
X_scaled = preprocessing.scale(X)
Y = data["Enrolment Rate, Tertiary (%)"]
model.fit(X_scaled, Y)

mse = np.square(Y - model.predict(X_scaled)).mean()
print("RMSE: " + str(np.sqrt(mse)))

plt.scatter(data["GDP per Capita (PPP USD)"], 
            data["Enrolment Rate, Tertiary (%)"])
plt.title('GDP vs Enrolment Rate')
plt.xlabel("GDP per Capita (PPP USD)")
plt.ylabel("Enrolment Rate, Tertiary (%)")

x_pred = data["GDP per Capita (PPP USD)"]
plt.plot(x_pred, model.predict(X_scaled), '.', color='orange')
plt.show()

headers=list(X)
coefficients = []
for i in range(len(headers)):
    coefficients.append({"Property": headers[i], "coefficient": model.coef_[i]})
pd.DataFrame(coefficients)

## Polynomial Regression

We can take our existing input features and have them transformed into a series of polynomial features.  

With degree 2, features $[a, b]$ would become $[1, a, b, a^2, ab, b^2]$.  
Our original 11 features become 78 polynomial features that are able to capture pairwise feature interactions.

The number of polynomial features can be estimated as follows:

<center>
$\begin{aligned}
\frac{(n + d)!}{n! d!}
\end{aligned}$ 
</center>

where $n$ is the original number of features, and $d$ is the polynomial degree. Thus, $\frac{(11 + 2)!}{11! 2!} = 78$ here.

Polynomial regression is a special case of multiple linear regression – as a statistical estimation problem it is linear, but because it uses polynomial features it is able to model non-linear data.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

model = LinearRegression(fit_intercept=True)
X = data.copy().drop(["Country Name", "Enrolment Rate, Tertiary (%)"], axis=1)
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)
Y = data["Enrolment Rate, Tertiary (%)"]
model.fit(X_poly, Y)

mse = np.square(Y - model.predict(X_poly)).mean()
print("RMSE: " + str(np.sqrt(mse)))


plt.scatter(data["GDP per Capita (PPP USD)"], 
            data["Enrolment Rate, Tertiary (%)"])
plt.title('GDP vs Enrolment Rate')
plt.xlabel("GDP per Capita (PPP USD)")
plt.ylabel("Enrolment Rate, Tertiary (%)")

x_pred = data["GDP per Capita (PPP USD)"]
plt.plot(x_pred, model.predict(X_poly), '.', color='orange')


plt.show()

Now let's try 3rd degree polynomial features. Our original 11 features are transformed into 364 features ($\frac{(11 + 3)!}{11! 3!} = 364$).

The resulting model captures the dataset almost perfectly.

In [ ]:
model = LinearRegression(fit_intercept=True)
X = data.copy().drop(["Country Name", "Enrolment Rate, Tertiary (%)"], axis=1)
poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(X)
print(X_poly.shape) #check the number of data points vs the number of features
Y = data["Enrolment Rate, Tertiary (%)"]
model.fit(X_poly, Y)

mse = np.square(Y - model.predict(X_poly)).mean()
print("RMSE: " + str(np.sqrt(mse)))

plt.scatter(data["GDP per Capita (PPP USD)"], 
            data["Enrolment Rate, Tertiary (%)"])
plt.title('GDP vs Enrolment Rate')
plt.xlabel("GDP per Capita (PPP USD)")
plt.ylabel("Enrolment Rate, Tertiary (%)")

x_pred = data["GDP per Capita (PPP USD)"]
plt.plot(x_pred, model.predict(X_poly), '.', color='orange')
plt.show()

In fact, this model captures the dataset too perfectly.  
There are twice as many features ($364$) for each datapoint as there are datapoints in the whole dataset ($161$). Such a model often has the capacity to memorize the training data, but that doesn't mean it is able to properly capture the underlying patterns.

## Dataset Splits for Model Selection

Let's try an experiment. We will split our available dataset into three parts: training, development and test set.  
We can fit the model on the **training** data, validate its performance and select model hyperparameters using the **development** data, and then finally evaluate the model using the **test** data.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

print(X.shape)
print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)

We can define a function for fitting polynomial regression with different degrees:

In [ ]:
def fit_polynomial_regression(X, Y, degree):
    model = LinearRegression(fit_intercept=True)
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)
    model.fit(X_poly, Y)
    return model

Let's define one more function, to help us evaluate and plot the output from trained polynomial models:

In [ ]:
def plot_gdp_vs_enrolment_rate(X, Y, degree, model):
    plt.scatter(X["GDP per Capita (PPP USD)"], Y)
    plt.title('GDP vs Enrolment Rate')
    plt.xlabel("GDP per Capita (PPP USD)")
    plt.ylabel("Enrolment Rate, Tertiary (%)")
    
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)
    
    mse = np.square(Y - model.predict(X_poly)).mean()
    print("RMSE: " + str(np.sqrt(mse)))
    
    x_pred = X["GDP per Capita (PPP USD)"]
    plt.plot(x_pred, model.predict(X_poly), '.', color='orange')

    plt.show()

First, let's try training the 3rd degree polynomial model on the training set and testing also on the training set:

In [ ]:
model = fit_polynomial_regression(X_train, y_train, degree=3)
plot_gdp_vs_enrolment_rate(X_train, y_train, degree=3, model=model)

Looks good, still a perfect fit.  

How about fitting the same model on the training set and then evaluating on the development set:

In [ ]:
model = fit_polynomial_regression(X_train, y_train, degree=3)
plot_gdp_vs_enrolment_rate(X_dev, y_dev, degree=3, model=model)

Well, that's no good. The predictions are all over the place.  
This model perfectly memorizes the training set but is not able to predict anything correctly on the development set. This might be a sign of **overfitting**. Let's try a lower-order model.

In [ ]:
model = fit_polynomial_regression(X_train, y_train, degree=2)
plot_gdp_vs_enrolment_rate(X_dev, y_dev, degree=2, model=model)

Actually better. Now let's go all the way down to the first degree model.

In [ ]:
model = fit_polynomial_regression(X_train, y_train, degree=1)
plot_gdp_vs_enrolment_rate(X_dev, y_dev, degree=1, model=model)

And this is indeed the model that gives the smallest error on held-out data.

If we had a much larger dataset, with thousands or tens of thousands of examples, then the model wouldn't be able to overfit to the training set so easily and the polynomial features might actually provide a benefit.

Also, when we have much fewer features available, the polynomial model can help model the correct shape in the data. We can try polynomial regression with only one variable again: the GDP.

In [ ]:
X_single = pd.DataFrame(data, columns=["GDP per Capita (PPP USD)"])
Y_single = data["Enrolment Rate, Tertiary (%)"]

X_train_single, X_test_single, y_train_single, y_test_single = train_test_split(X_single, 
                                                                                Y_single, 
                                                                                test_size=0.2, 
                                                                                random_state=1)
X_train_single, X_dev_single, y_train_single, y_dev_single = train_test_split(X_train_single, 
                                                                              y_train_single, 
                                                                              test_size=0.2, 
                                                                              random_state=1)

model = fit_polynomial_regression(X_train_single.values.reshape(-1,1), y_train_single, degree=3)
plot_gdp_vs_enrolment_rate(X_train_single, y_train_single, degree=3, model=model)
plot_gdp_vs_enrolment_rate(X_dev_single, y_dev_single, degree=3, model=model)

In [ ]:
plt.scatter(data["GDP per Capita (PPP USD)"], 
            data["Enrolment Rate, Tertiary (%)"])
plt.title('GDP vs Enrolment Rate')
plt.xlabel("GDP per Capita (PPP USD)")
plt.ylabel("Enrolment Rate, Tertiary (%)")

countries = list(data["Country Name"])
for c in ["UK", "USA", "South Korea", "Luxembourg", "Qatar", "Malawi"]:
    country_num = countries.index(c)
    plt.annotate(countries[country_num], (data["GDP per Capita (PPP USD)"][country_num], 
                                          data["Enrolment Rate, Tertiary (%)"][country_num]))

plt.show()

Tertiary education numbers can be checked here: http://wdi.worldbank.org/table/2.8

## How to tell that a model is overfitting?

Take a look at how the RMSE on the training and the development sets change with the number of examples:

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def plot_learning_curves(X_train, y_train, X_dev, y_dev, degree, ylim_range):
    pipeline = Pipeline((
                        ("scaler", MinMaxScaler()),
                        ("poly", PolynomialFeatures(degree=degree)),
                        ("model", LinearRegression()),
                        ))
    train_errors = []
    dev_errors = []
    for m in range(1, len(X_train)):
        pipeline.fit(X_train[:m], y_train[:m])
        y_train_pred = pipeline.predict(X_train[:m])
        y_dev_pred = pipeline.predict(X_dev)
        train_errors.append(np.sqrt(mean_squared_error(y_train_pred, y_train[:m])))
        dev_errors.append(np.sqrt(mean_squared_error(y_dev_pred, y_dev)))
        #train_errors.append(np.sqrt(np.square(y_train[:m] - y_train_pred).mean()))
        #dev_errors.append(np.sqrt(np.square(y_dev - y_dev_pred).mean()))                                  
    #print(dev_errors)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.ylim(0, ylim_range)
    ax.plot(train_errors, "+-", color='darkblue', linewidth=2, label="train")
    ax.plot(dev_errors, 'x-', color='darkorange', linewidth=2, label="dev")
    ax.legend(loc="upper right", fontsize=14)
    ax.set_xlabel('Training set size')
    ax.set_ylabel('RMSE')
    plt.show()

Explore what happens with the simple regression model:

In [ ]:
plot_learning_curves(X_train, y_train, X_dev, y_dev, 1, 100)

Note how even though the error reaches a plateau on the training data, it is never exactly zero, i.e. the model doesn't catpure the training data perfectly. This is because the data is not exactly linear and it is noisy. At the same time, the error on the deevelopment set keeps dropping with more examples until it approximates the error on the training set quite closely.

Now what happens if you plot the errors of the polynomial model?

In [ ]:
plot_learning_curves(X_train, y_train, X_dev, y_dev, 3, 100)

Now the error on the training data is around zero (the training data is modelled, or rather memorized, perfectly well), but the difference between the training and development data is significant (note the difference in scale of errors).

## Regularization

- Ridge Regression cost function:

<center>
$\begin{aligned}
J(\theta) = MSE(\theta) + \alpha \frac{1}{2} \sum_{i=1}^{n} \theta_i^{2}
\end{aligned}$ 
</center>


- Lasso Regression cost function:

<center>
$\begin{aligned}
J(\theta) = MSE(\theta) + \alpha \sum_{i=1}^{n} |\theta_i |
\end{aligned}$ 
</center>




- Elastic Net:

<center>
$\begin{aligned}
J(\theta) = MSE(\theta) + r\alpha \sum_{i=1}^{n} |\theta_i | + \frac{1-r}{2}\alpha\sum_{i=1}^{n} \theta_i^{2}
\end{aligned}$ 
</center>


